In [ ]:
import pandas as pd # library for the DataFrame and data analsysis
pd.set_option('display.max_columns', None) #
pd.set_option('display.max_row', None) #

import numpy  as np # library for mathematical calculations of data, the build of arrays and vectorization

import json # library to handle JSON file
import requests #library to handle requests

#conda install -c conda-forge geopy --yes #
from geopy.geocoders import Nominatim #convert an address into latitude and longitude values


from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#Matplotlib and associated plotting modules
import matplotlib.cm as cm 
import matplotlib.colors as colors 

#For web Scrapingf
from bs4 import BeautifulSoup 

# for the clustering data with Kmean model
from sklearn.cluster import KMeans #

#!conda install -c conda-forge folium=0.5.0 --yes #
import folium # map rendering librar
print('Libraries imported')

Libraries imported


# PART A): 
#Exploring and clustering of  neighborhoods of Toronto into DataFrame by webscraping of wikipedia website

#### Recovery of the URL, Scraping of the wikipedia site and creation of a "Beautiful soup" object.

In [ ]:
url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
rq = requests.get(url).text
soup = BeautifulSoup(rq, 'html5lib')

### Find postal code by Borough and Neighbourhood to Canada(Toronto...) and includ that into a DataFrame, Remove "Not assiged",  and agregate "Harbourfront and Regent Park " into one row of codepostal M5A

In [ ]:
# creating list
table_contents=[]
#table and table data
table=soup.find('table')
for row in table.findAll('td'):
#create a dictionary called cell and Remove "Not assigned"
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3] # extract postal code using tablerow.p.text

        # getting Borough and Neighborhood information by use split ,strip and replace functions
        cell['Borough'] = (row.span.text).split('(')[0] 
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ') 
        table_contents.append(cell) # Append to the list

print(table_contents)

[{'PostalCode': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'}, {'PostalCode': 'M4A', 'Borough': 'North York', 'Neighborhood': 'Victoria Village'}, {'PostalCode': 'M5A', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Regent Park, Harbourfront'}, {'PostalCode': 'M6A', 'Borough': 'North York', 'Neighborhood': 'Lawrence Manor, Lawrence Heights'}, {'PostalCode': 'M7A', 'Borough': "Queen's Park", 'Neighborhood': 'Ontario Provincial Government'}, {'PostalCode': 'M9A', 'Borough': 'Etobicoke', 'Neighborhood': 'Islington Avenue'}, {'PostalCode': 'M1B', 'Borough': 'Scarborough', 'Neighborhood': 'Malvern, Rouge'}, {'PostalCode': 'M3B', 'Borough': 'North York', 'Neighborhood': 'Don Mills North'}, {'PostalCode': 'M4B', 'Borough': 'East York', 'Neighborhood': 'Parkview Hill, Woodbine Gardens'}, {'PostalCode': 'M5B', 'Borough': 'Downtown Toronto', 'Neighborhood': 'Garden District, Ryerson'}, {'PostalCode': 'M6B', 'Borough': 'North York', 'Neighborhood': 'Glencairn'}, {'PostalCode': 'M9

### Create a dataframe with list

In [23]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head(18)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### Size of Row and columns to the DataFrame

In [24]:
print(df.shape)

(103, 3)


# **PART B):** Read Content Geospacial Coordinates - CSV file- into a DataFrame

---



---



In [29]:
# Read the csv file containing only postal codes and geospatial coordinates
dfg = pd.read_csv('/content/Geospatial_Coordinates.csv') 
# Creating variable names for dfg
dfg.columns = ['PostalCode', 'Latitude', 'Longitude']

dfg.head(16)

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [30]:
#Copy missing variables (Borhough and Neighborhood) into dfg from df1
df1 = pd.merge(df, dfg, on=['PostalCode'], how='inner')
df2 = df1[['PostalCode','Borough', 'Neighborhood', 'Latitude', 'Longitude']].copy()
df2.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
